In [7]:
import numpy as np
import pandas as pd
import warnings
import os
from tqdm import tqdm
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
from collections import Counter
import time 
import datetime
import scipy
import matplotlib 
from sklearn.utils import shuffle
import sklearn
from sklearn import preprocessing, metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle# 随机排列

from sklearn import preprocessing
import gc
import xgboost as xgb 
import joblib

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
warnings.filterwarnings('ignore')

In [8]:
feature_train=pd.read_pickle('../user_data/tmp_data/feature_train.pkl')
feature_test=pd.read_pickle('../user_data/tmp_data/feature_test.pkl')

feature_train=shuffle(feature_train)

In [9]:
feature_test.head().T

,0,1,2,3,4
majorid,20200301100002543_3,20200301100002543_3,20200301100002543_3,20200301100002543_3,20200301100006950_3
courier_id,100002543,100002543,100002543,100002543,100006950
wave_index,3,3,3,3,3
date,20200301,20200301,20200301,20200301,20200301
tracking_id,2100076045156320581,2100076044413207877,2100076045016368756,2100076044139038021,2100076085507393263
action_type,DELIVERY,PICKUP,DELIVERY,DELIVERY,DELIVERY
last_tracking_id,2100076044139038021,2100076044139038021,2100076044139038021,2100076044139038021,2100076086461237066
last_action_type,PICKUP,PICKUP,PICKUP,PICKUP,PICKUP
load,3,3,3,3,2
old_action_1,PICKUP,PICKUP,PICKUP,PICKUP,PICKUP


In [10]:
feature_train.head().T

,78839,176383,141116,10250,158973
majorid,20200214116133289_1,2020022710774871_1,20200223102437106_8,20200202118890377_5,2020022510649045_1
courier_id,116133289,10774871,102437106,118890377,10649045
wave_index,1,1,8,5,1
date,20200214,20200227,20200223,20200202,20200225
tracking_id,2100075082385261678,2100075812186982202,2100075581498557681,2100074624256410839,2100075669234484811
action_type,DELIVERY,PICKUP,PICKUP,DELIVERY,DELIVERY
last_tracking_id,2100075081730852630,2100075805299770805,2100075578441729413,2100074623223629076,2100075671310337801
last_action_type,PICKUP,DELIVERY,PICKUP,PICKUP,PICKUP
load,4,0,2,2,5
old_action_1,PICKUP,DELIVERY,PICKUP,PICKUP,PICKUP


In [11]:
prediction = feature_test[['courier_id', 'wave_index', 'tracking_id', 
'courier_wave_start_lng', 'courier_wave_start_lat', 
'action_type', 'expect_time', 'date', 'majorid', 'label']]
prediction['label'] = 0

y_col = 'label'
# x_col =['majorid', 'courier_id', 'wave_index', 'date', 'tracking_id',
    #    'action_type', 'last_tracking_id', 'last_action_type', 'load',
    #    'old_action_1', 'old_action_2', 'grid_distance', 'target_tan',
    #    'target_MHD', 'level', 'speed', 'max_load', 'weather_grade',
    #    'delivery_tan', 'delivery_MHD', 'last_delivery_tan',
    #    'last_delivery_MHD', 'assigned_time', 'promise_deliver_time',
    #    'estimate_pick_time', 'delta_time_pc', 'last_assigned_time',
    #    'last_promise_deliver_time', 'last_estimate_pick_time',
    #    'last_delta_time_pc', 'last_time', 'expect_time', 'label',
    #    'courier_wave_start_lng', 'courier_wave_start_lat', 'now_DELIVERY',
    #    'now_PICKUP', 'last_DELIVERY', 'last_PICKUP', 'old1_DELIVERY',
    #    'old1_PICKUP', 'old2_DELIVERY', 'old2_PICKUP', 'normal', 'a_bit_bad',
    #    'bad', 'vary_bad']
x_col = ['load','grid_distance', 'target_tan', 'target_MHD', 'level', 'speed', 'max_load']

model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    # class_weight="is_unbalance",
                           num_leaves=50,
                           max_depth=7,
                           learning_rate=0.03,
                           n_estimators=10000,
                           subsample=0.8,
                           feature_fraction=0.8,
                           reg_alpha=0.8,
                           reg_lambda=0.8,
                           random_state=220,
                           metric=None,
                           n_jobs=-1
                           )

In [12]:
oof = []
df_importance_list = []
n=5
kfold = GroupKFold(n_splits=n)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(feature_train[x_col], feature_train[y_col], feature_train['majorid'])):
    X_train = feature_train.iloc[trn_idx][x_col]
    Y_train = feature_train.iloc[trn_idx][y_col]

    X_val = feature_train.iloc[val_idx][x_col]
    Y_val = feature_train.iloc[val_idx][y_col]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    lgb_model = model.fit(
        X_train,
        Y_train,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, Y_train), (X_val, Y_val)],
        verbose=200,
        eval_metric='auc',
        early_stopping_rounds=200
    )

    pred_val = lgb_model.predict_proba(X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    df_oof = feature_train.iloc[val_idx][["courier_id", 'majorid', y_col]].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = lgb_model.predict_proba(feature_test[x_col], num_iteration=lgb_model.best_iteration_)[:, 1]
    prediction['label'] += pred_test / n

    df_importance = pd.DataFrame({
        'column': x_col,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)


Fold_1 Training ================================

Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.735093	valid's auc: 0.725382
Early stopping, best iteration is:
[167]	train's auc: 0.733756	valid's auc: 0.725472

Fold_2 Training ================================

Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.734827	valid's auc: 0.725629
[400]	train's auc: 0.742455	valid's auc: 0.725327
Early stopping, best iteration is:
[215]	train's auc: 0.735442	valid's auc: 0.725655

Fold_3 Training ================================

Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.733926	valid's auc: 0.72924
[400]	train's auc: 0.741647	valid's auc: 0.728985
Early stopping, best iteration is:
[207]	train's auc: 0.7342	valid's auc: 0.729261

Fold_4 Training ================================

Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.735144	valid's auc: 0.724

In [13]:
feature_test.head()

,majorid,courier_id,wave_index,date,tracking_id,action_type,last_tracking_id,last_action_type,load,old_action_1,old_action_2,grid_distance,target_tan,target_MHD,level,speed,max_load,weather_grade,delivery_tan,delivery_MHD,last_delivery_tan,last_delivery_MHD,assigned_time,promise_deliver_time,estimate_pick_time,delta_time_pc,last_assigned_time,last_promise_deliver_time,last_estimate_pick_time,last_delta_time_pc,last_time,expect_time,label,courier_wave_start_lng,courier_wave_start_lat,now_DELIVERY,now_PICKUP,last_DELIVERY,last_PICKUP,old1_DELIVERY,old1_PICKUP,old2_DELIVERY,old2_PICKUP,normal,a_bit_bad,bad,vary_bad
0,20200301100002543_3,100002543,3,20200301,2100076045156320581,DELIVERY,2100076044139038021,PICKUP,3,PICKUP,PICKUP,7.765145,1.642096,5.768373,3,5.536055,10,1,1.658933,5.768373,1.563800,6.129087,1583036629,1583039154,1583037835,0.440905,1583037289,1583040204,1583038706,0.289873,1583038335,0,None,121.634234,39.143629,1,0,0,1,0,1,0,1,1,0,0,0
1,20200301100002543_3,100002543,3,20200301,2100076044413207877,PICKUP,2100076044139038021,PICKUP,3,PICKUP,PICKUP,0.693147,NaN,0.000000,3,5.536055,10,1,1.714542,5.480707,1.563800,6.129087,1583036809,1583039641,1583038022,0.475031,1583037289,1583040204,1583038706,0.289873,1583038335,0,None,121.634234,39.143629,0,1,0,1,0,1,0,1,1,0,0,0
2,20200301100002543_3,100002543,3,20200301,2100076045016368756,DELIVERY,2100076044139038021,PICKUP,3,PICKUP,PICKUP,8.217439,1.820522,4.787734,3,5.536055,10,1,-1.872481,4.381142,1.563800,6.129087,1583037829,1583040530,1583038430,0.488795,1583037289,1583040204,1583038706,0.289873,1583038335,0,None,121.634234,39.143629,1,0,0,1,0,1,0,1,1,0,0,0
3,20200301100002543_3,100002543,3,20200301,2100076044139038021,DELIVERY,2100076044139038021,PICKUP,3,PICKUP,PICKUP,7.805475,1.549883,6.129087,3,5.536055,10,1,1.565636,6.129087,1.563800,6.129087,1583037289,1583040204,1583038706,0.481743,1583037289,1583040204,1583038706,0.289873,1583038335,0,None,121.634234,39.143629,1,0,0,1,0,1,0,1,1,0,0,0
4,20200301100006950_3,100006950,3,20200301,2100076085507393263,DELIVERY,2100076086461237066,PICKUP,2,PICKUP,DELIVERY,7.294377,0.102299,7.348138,2,5.108438,6,1,0.157160,7.399583,0.327814,7.311422,1583060079,1583063178,1583061259,0.509157,1583060499,1583063341,1583061483,0.279538,1583061548,0,None,121.412991,39.032091,1,0,0,1,0,1,1,0,1,0,0,0


In [14]:
prediction.shape

(13322, 10)

In [15]:
prediction["expect_time"]=feature_test["last_time"]+100

In [16]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,grid_distance,2381.8
1,target_tan,2103.6
2,target_MHD,1825.4
3,speed,1789.4
4,load,1345.6
5,max_load,746.4
6,level,254.6


In [17]:
def wave_label_func(group):
    target_list = group['label'].values.tolist()
    pred_list = group['pred'].values.tolist()
    max_index = pred_list.index(max(pred_list))
    if target_list[max_index] == 1:
        return 1
    else:
        return 0

In [18]:
df_oof = pd.concat(oof)
df_temp = df_oof.groupby(['majorid']).apply(wave_label_func).reset_index()
df_temp.columns = ['majorid', 'label']
acc = df_temp[df_temp['label'] == 1].shape[0] / df_temp.shape[0]
print('acc:', acc)

acc: 0.7577453866938073


In [19]:
def result_func(majorid):
    majorid = majorid.values.tolist()
    max_index = majorid.index(max(majorid))
    result = np.zeros(len(majorid))
    result[max_index] = 1
    return result

In [20]:
prediction['rusult'] = prediction.groupby(['majorid'])['label'].transform(result_func)
subfile = prediction[prediction['rusult'] == 1]

In [21]:
result = subfile[['courier_id', 'wave_index', 'tracking_id', 'courier_wave_start_lng', 'courier_wave_start_lat', 'action_type', 'expect_time', 'date']]

In [22]:
# import zipfile
# os.makedirs('../action_predict/{}'.format('result'), exist_ok=True)
# f = zipfile.ZipFile('../action_predict/{}.zip'.format('result'), 'w', zipfile.ZIP_DEFLATED)
for date in result['date'].unique():
    df_temp = result[prediction['date'] == date]
    # dict_action={0:None,1:'PICKUP',2:"DELIVERY"}
    # df_temp["action_type"]=df_temp["action_type"].map(dict_action)
    # assert False
    df_temp["courier_wave_start_lng"]=df_temp["courier_wave_start_lng"].round(6)
    df_temp["courier_wave_start_lat"]=df_temp["courier_wave_start_lat"].round(6)
    del df_temp['date']
    df_temp.to_csv('../action_predict/action_{}.txt'.format(date), index=False)
    # f.write('../action_predict/action_{}.txt'.format(date), 'action_{}.txt'.format(date))
# f.close()

In [23]:
df_temp.head(20)

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time
10414,100002543,0,2100076472460444744,121.644470,39.135076,PICKUP,1583467132
10417,100002652,0,2100076483679979410,121.559951,39.151688,PICKUP,1583471189
10420,100002652,2,2100076520321943315,121.580403,39.151811,PICKUP,1583492977
10422,100006950,2,2100076524356470084,121.405090,39.043030,DELIVERY,1583493871
10426,100084429,0,2100076459080353111,121.255337,38.948718,DELIVERY,1583465582
10427,100084429,2,2100076481663370502,121.262100,38.956129,PICKUP,1583470812
10430,10012191,0,2100076471646848007,121.521817,39.206101,PICKUP,1583467947
10433,10012191,2,2100076484725442041,121.530141,39.201457,DELIVERY,1583474668
10439,100122337,0,2100076455181616167,121.593730,39.203980,PICKUP,1583466700
10443,100122337,2,2100076502798337304,121.580896,39.211271,PICKUP,1583486519
